In [1]:
#Install nltk
#!conda install -c anaconda nltk
import nltk
#nltk.download ()

#https://pythonspot-9329.kxcdn.com/wp-content/uploads/2016/08/nltk-speech-codes.png

import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score
import pickle
import numpy as np
from keras.models import model_from_json

#!conda install -n mldds -c anaconda joblib
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Cores: ", num_cores)

import time
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': num_cores} )
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from MyTotoResearchv4 import *

%matplotlib inline
#Install nltk
#!conda install -c anaconda nltk
import nltk
#nltk.download ()

#https://pythonspot-9329.kxcdn.com/wp-content/uploads/2016/08/nltk-speech-codes.png
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
import pickle

class FBModel:
    def __init__(self):
        np.random.seed(42)
        pass
    
    def get_df(self):
        return self.df
    
    def preprocess (self, df):
        self.df = df 
        #nans = lambda df: df[df.isnull().any(axis=1)]
        #show rows which has nan in any columns
        #nans(df)
        
        #delete missing messages
        delete_rows = df[df['message'].isna()].index
        df.drop(index=delete_rows, inplace=True)
        df['click_rate'] = df['clicks_unique'] / (df['impressions_organic_unique'] + df['impressions_paid_unique'])
        df['reaction_rate'] = (df['reactions_haha'] + df['reactions_like'] + df['reactions_love'] + df['reactions_sorry']) / (df['impressions_organic_unique'] + df['impressions_paid_unique'])
        df['engagement_rate'] = df['engagements'] / (df['impressions_organic_unique'] + df['impressions_paid_unique'])
        df['comment_rate'] = df['comments'] / (df['impressions_organic_unique'] + df['impressions_paid_unique'])
        df['shares_rate'] = df['shares'] / (df['impressions_organic_unique'] + df['impressions_paid_unique'])

        self.encode_features(df)
        
    def encode_features(self, df):
        cols_to_encode = ['amplification', 'intent', 'type', 'weekday', 'message_length']
        self.encoders = {}
        for i in range(0, len(cols_to_encode)):
            col = cols_to_encode[i]
            self.encoders[col] = LabelEncoder()
            df[col + '_encoded'] = self.encoders[col].fit_transform(df[col])
        
    def scale_features(self, X):
        self.scaler = StandardScaler()
        self.scaler.fit(X)
        return self.scaler.transform(X)
    
    def get_features_to_use(self, df):
        features = [
            'amplification_encoded', 
#    'intent_encoded', 
            'type_encoded', 
#     'last_post_time_diff',
#    'time', 
            'weekday',
#    'num_creatives',
            'message_length',
        ]
        return df[features]
    
    
    def save_model(self, model, file_name_without_extension):
        model_json = model.to_json()
        with open(file_name_without_extension + ".json", "w") as json_file:
            json_file.write(model_json)
            # serialize weights to HDF5
        model.save_weights(file_name_without_extension + ".h5")
        print("Saved model to disk")
        pickle.dump(self.scaler, open(file_name_without_extension + '-scaler.pkl', 'wb'))
        pickle.dump(self.encoders, open(file_name_without_extension + '-encoders.pkl', 'wb'))


    def load_model(self, file_name_without_extension):
        # fix random seed for reproducibility
        # load json and create model
        json_file = open(file_name_without_extension + '.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights(file_name_without_extension + ".h5")
        self.scaler_loaded = pickle.load(open(file_name_without_extension + '-scaler.pkl', 'rb'))
        self.encoders_loaded = pickle.load(open(file_name_without_extension + '-encoders.pkl', 'rb'))
        return model
    
    def get_message_doc(self, messages):
        # integer encode the messages
        vocab_size = 50
        encoded_docs = [one_hot(d, vocab_size) for d in messages]

        # pad messages to a max length of 4 words
        max_length = 3
        return  pad_sequences(encoded_docs, maxlen=max_length, padding='post')


    def predict ( self, df, file_name_without_extension ):
        model = self.load_model(file_name_without_extension)
        cols_to_encode = ['amplification', 'intent', 'type']        
        for col in cols_to_encode:
            df[col + '_encoded'] = self.encoders_loaded[col].transform(df[col])
        X = self.get_features_to_use(df)
        X_scaled = self.scaler_loaded.transform(X)
        padded_docs = self.get_message_doc(df['message'].values)
        return model.predict([padded_docs,X_scaled])
        

Using TensorFlow backend.


Cores:  12
Done.


In [2]:
url = 'posts.json'
df = pd.read_json(url, orient='columns')

model_class = FBModel()
model_class.preprocess(df)

X = model_class.get_features_to_use(df)

#X['message'] = df['message']
y = df['shares_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Z_train = model_class.scale_features(X_train)
Z_test = model_class.scaler.transform(X_test)


/Users/uqapp/anaconda3/envs/mldss/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/uqapp/anaconda3/envs/mldss/lib/python3.6/site-packages/ipykernel_launcher.py:103: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
/Users/uqapp/anaconda3/envs/mldss/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [3]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam, Adagrad, SGD
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.optimizers import Adagrad
from sklearn.preprocessing import StandardScaler


In [4]:
# X_train_message = df.loc[X_train.index]['message']
# X_test_message = df.loc[X_test.index]['message']


# integer encode the messages
vocab_size = 50
max_length = 3

padded_docs = model_class.get_message_doc(df.loc[X_train.index]['message'].values)
padded_docs_test = model_class.get_message_doc(df.loc[X_test.index]['message'].values)

# first_input = Input(shape=(max_length, ))
# first_dense = Embedding(vocab_size, 8)(first_input)
# first_flattened = Flatten()(first_dense)

second_input = Input(shape=(Z_train.shape[1], ))
second_dense = Dense(4, activation='relu')(second_input)

# merge_one = concatenate([first_flattened, second_dense])
# third_dense = Dense(2, activation='relu')(merge_one)

third_dense = Dense(2, activation='relu')(second_dense)

output = Dense(1)(third_dense)

#model = Model(inputs=[first_input, second_input], outputs=output)

model = Model(inputs=[second_input], outputs=output)
optimizer = Adam(1e-4)
sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer='Adagrad', loss='mae', metrics=['mae'])
model.summary()
history = model.fit(Z_train, y_train, epochs=4500, validation_data=(Z_test,y_test), batch_size=batch_size, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


NameError: name 'batch_size' is not defined

In [ ]:
#!pip install pydot
from keras.utils.vis_utils import plot_model  
plot_model(model, to_file='fb_model.png')

from IPython.display import Image
Image(filename='fb_model.png')

In [5]:
batch_size = 128
#history = model.fit([padded_docs, Z_train], y_train, epochs=450, validation_data=([padded_docs_test,X_test],y_test), batch_size=batch_size, verbose=1)
history = model.fit(Z_train, y_train, epochs=4500, validation_data=(X_test,y_test), batch_size=batch_size, verbose=1)

Train on 469 samples, validate on 232 samples
Epoch 1/4500
469/469 [==============================] - 0s 933us/step - loss: 0.2639 - mean_absolute_error: 0.2639 - val_loss: 100.3212 - val_mean_absolute_error: 100.3212
Epoch 2/4500
469/469 [==============================] - 0s 18us/step - loss: 0.1815 - mean_absolute_error: 0.1815 - val_loss: 90.3955 - val_mean_absolute_error: 90.3955
Epoch 3/4500
469/469 [==============================] - 0s 18us/step - loss: 0.1457 - mean_absolute_error: 0.1457 - val_loss: 82.8172 - val_mean_absolute_error: 82.8172
Epoch 4/4500
469/469 [==============================] - 0s 20us/step - loss: 0.1207 - mean_absolute_error: 0.1207 - val_loss: 76.6780 - val_mean_absolute_error: 76.6780
Epoch 5/4500
469/469 [==============================] - 0s 18us/step - loss: 0.1015 - mean_absolute_error: 0.1015 - val_loss: 71.5746 - val_mean_absolute_error: 71.5746
Epoch 6/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0860 - mean_absolute_error:

Epoch 49/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0059 - mean_absolute_error: 0.0059 - val_loss: 16.9004 - val_mean_absolute_error: 16.9004
Epoch 50/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0058 - mean_absolute_error: 0.0058 - val_loss: 16.7828 - val_mean_absolute_error: 16.7828
Epoch 51/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0058 - mean_absolute_error: 0.0058 - val_loss: 16.6761 - val_mean_absolute_error: 16.6761
Epoch 52/4500
469/469 [==============================] - 0s 19us/step - loss: 0.0055 - mean_absolute_error: 0.0055 - val_loss: 16.5754 - val_mean_absolute_error: 16.5754
Epoch 53/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0054 - mean_absolute_error: 0.0054 - val_loss: 16.4374 - val_mean_absolute_error: 16.4374
Epoch 54/4500
469/469 [==============================] - 0s 16us/step - loss: 0.0052 - mean_absolute_error: 0.0052 - val_loss: 16.3097 - val_mean_abso

469/469 [==============================] - 0s 19us/step - loss: 0.0021 - mean_absolute_error: 0.0021 - val_loss: 13.6280 - val_mean_absolute_error: 13.6280
Epoch 98/4500
469/469 [==============================] - 0s 22us/step - loss: 0.0021 - mean_absolute_error: 0.0021 - val_loss: 13.6345 - val_mean_absolute_error: 13.6345
Epoch 99/4500
469/469 [==============================] - 0s 19us/step - loss: 0.0021 - mean_absolute_error: 0.0021 - val_loss: 13.6117 - val_mean_absolute_error: 13.6117
Epoch 100/4500
469/469 [==============================] - 0s 20us/step - loss: 0.0020 - mean_absolute_error: 0.0020 - val_loss: 13.6184 - val_mean_absolute_error: 13.6184
Epoch 101/4500
469/469 [==============================] - 0s 16us/step - loss: 0.0020 - mean_absolute_error: 0.0020 - val_loss: 13.6251 - val_mean_absolute_error: 13.6251
Epoch 102/4500
469/469 [==============================] - 0s 18us/step - loss: 0.0020 - mean_absolute_error: 0.0020 - val_loss: 13.6240 - val_mean_absolute_error:

469/469 [==============================] - 0s 16us/step - loss: 0.0014 - mean_absolute_error: 0.0014 - val_loss: 13.6230 - val_mean_absolute_error: 13.6230
Epoch 146/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0013 - mean_absolute_error: 0.0013 - val_loss: 13.6230 - val_mean_absolute_error: 13.6230
Epoch 147/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0013 - mean_absolute_error: 0.0013 - val_loss: 13.6231 - val_mean_absolute_error: 13.6231
Epoch 148/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0013 - mean_absolute_error: 0.0013 - val_loss: 13.6233 - val_mean_absolute_error: 13.6233
Epoch 149/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0014 - mean_absolute_error: 0.0014 - val_loss: 13.6233 - val_mean_absolute_error: 13.6233
Epoch 150/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0014 - mean_absolute_error: 0.0014 - val_loss: 13.6232 - val_mean_absolute_erro

Epoch 193/4500
469/469 [==============================] - 0s 16us/step - loss: 0.0012 - mean_absolute_error: 0.0012 - val_loss: 13.6219 - val_mean_absolute_error: 13.6219
Epoch 194/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0013 - mean_absolute_error: 0.0013 - val_loss: 13.6221 - val_mean_absolute_error: 13.6221
Epoch 195/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0012 - mean_absolute_error: 0.0012 - val_loss: 13.6223 - val_mean_absolute_error: 13.6223
Epoch 196/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0013 - mean_absolute_error: 0.0013 - val_loss: 13.6218 - val_mean_absolute_error: 13.6218
Epoch 197/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0012 - mean_absolute_error: 0.0012 - val_loss: 13.6217 - val_mean_absolute_error: 13.6217
Epoch 198/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0012 - mean_absolute_error: 0.0012 - val_loss: 13.6211 - val_mea

...

469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute_error: 13.5672
Epoch 4283/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute_error: 13.5672
Epoch 4284/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute_error: 13.5672
Epoch 4285/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute_error: 13.5672
Epoch 4286/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute_error: 13.5672
Epoch 4287/4500
469/469 [==============================] - 0s 13us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5672 - val_mean_absolute

Epoch 4330/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5669 - val_mean_absolute_error: 13.5669
Epoch 4331/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5670 - val_mean_absolute_error: 13.5670
Epoch 4332/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5669 - val_mean_absolute_error: 13.5669
Epoch 4333/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5669 - val_mean_absolute_error: 13.5669
Epoch 4334/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5670 - val_mean_absolute_error: 13.5670
Epoch 4335/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5669 - v

469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5666 - val_mean_absolute_error: 13.5666
Epoch 4378/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5665 - val_mean_absolute_error: 13.5665
Epoch 4379/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5666 - val_mean_absolute_error: 13.5666
Epoch 4380/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5665 - val_mean_absolute_error: 13.5665
Epoch 4381/4500
469/469 [==============================] - 0s 22us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5666 - val_mean_absolute_error: 13.5666
Epoch 4382/4500
469/469 [==============================] - 0s 18us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5665 - val_mean_absolute

Epoch 4425/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5662 - val_mean_absolute_error: 13.5662
Epoch 4426/4500
469/469 [==============================] - 0s 17us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5662 - val_mean_absolute_error: 13.5662
Epoch 4427/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5662 - val_mean_absolute_error: 13.5662
Epoch 4428/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5661 - val_mean_absolute_error: 13.5661
Epoch 4429/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5662 - val_mean_absolute_error: 13.5662
Epoch 4430/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5661 - v

469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute_error: 13.5658
Epoch 4473/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute_error: 13.5658
Epoch 4474/4500
469/469 [==============================] - 0s 16us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute_error: 13.5658
Epoch 4475/4500
469/469 [==============================] - 0s 15us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute_error: 13.5658
Epoch 4476/4500
469/469 [==============================] - 0s 14us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute_error: 13.5658
Epoch 4477/4500
469/469 [==============================] - 0s 13us/step - loss: 0.0011 - mean_absolute_error: 0.0011 - val_loss: 13.5658 - val_mean_absolute

In [ ]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
model_class.save_model(model, 'FB-Model-01')

In [ ]:
url = 'posts-predict.json'
df_predict = pd.read_json(url, orient='columns')

model_class = FBModel()
model_class.preprocess(df_predict)

y_pred = model_class.predict(df_predict,'FB-Model-01')
print(df_predict['shares_rate'].values)
print(y_pred)

In [ ]:
#END